In [26]:
import pandas as pd

df_NLP = pd.read_excel('final_project.ods', engine='odf', dtype=str)
df_NLP.sample(7)

,title,location,description,function,industry,career_level
7277,Senior Consultant Product Development Process ...,"Atlanta, GA",Execution of projects in various target indust...,consulting,Business Consulting,senior_specialist_or_project_manager
7443,"Manager, Solution Delivery","Wilton, CT",Provide managerial direction for application d...,information_technology_telecommunications,Property and Casualty Insurance,manager_team_leader
417,Director Program Engineering - Phoenix,"Phoenix, AZ",This is a high profile position responsible fo...,research_development,Air and Space Transport,manager_team_leader
5243,"Senior Manager, Systems Operations","San Diego, CA",Provide strategic information technology visio...,information_technology_telecommunications,Minimal-Invasive Surgery and Intervention,bereichsleiter
6251,Senior System Administrator,"San Mateo, CA",Composite Software is hiring a full-time Senio...,information_technology_telecommunications,Software Companies,senior_specialist_or_project_manager
4364,Regional Sales Manager - Life Sciences,Pennsylvania,ATMI is seeking a Regional Sales Manager.Â Th...,sales,Semiconductor Production,senior_specialist_or_project_manager
3791,"Manager, Visual Merchandising & Retail Environ...","Carlsbad, CA",JOB OBJECTIVE:Â Lead and manage US in-store m...,marketing,"Sports, Games, Hobbies",manager_team_leader


## Kiểm tra số class khác nhau trong từng features

In [27]:
df_NLP.nunique()

title           6790
location        1081
description     7973
function          19
industry         352
career_level       6
dtype: int64

In [28]:


df_NLP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         8074 non-null   object
 1   location      8074 non-null   object
 2   description   8073 non-null   object
 3   function      8074 non-null   object
 4   industry      8074 non-null   object
 5   career_level  8074 non-null   object
dtypes: object(6)
memory usage: 378.6+ KB


In [36]:
# from sklearn.impute import SimpleImputer

# df_NLP["description"] = SimpleImputer(strategy='most_frequent')
df_NLP.dropna(subset=['description'], inplace=True)
df_NLP.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8073 entries, 0 to 8073
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         8073 non-null   object
 1   location      8073 non-null   object
 2   description   8073 non-null   object
 3   function      8073 non-null   object
 4   industry      8073 non-null   object
 5   career_level  8073 non-null   object
dtypes: object(6)
memory usage: 441.5+ KB


# MỤC tiêu:
- tôi sẽ sử dụng tfidf(Term Frequency- Inverse Document Frequency) để tính điểm cho các cột : title, description
- sử dụng Onehotencoder : location, function, industry

**Dường như cột location khá nhiều giá trị khác nhau tuy nhiên sự sai khác đó đều cùng một dạng TENTHANHPHO, MABANG vì thế mong muốn của tôi là sử dụng regular để đưa chúng về cùng một dạng, việc này sẽ giúp giảm số lượng các giá trị khác nhau trong cột**

**Dự đoán của tôi có lẽ mô hình sẽ chạy khá lâu khi mã hóa cột description chính vì thế cần một số thao tác để xử lý cột này**

In [37]:
from sklearn.model_selection import train_test_split
import re


def string_format(x):
    results = re.findall('\,\ [A-Z]{2}$', x)
    if len(results) > 0:
        return results[0][2]
    else:
        return x
df_NLP["location"] = df_NLP["location"].apply(string_format) # 66 < 1081
# print(len(df_NLP["location"].unique()))

target = "career_level"
X = df_NLP.drop(target, axis=1)
y = df_NLP[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100, stratify=y) 
# bạn phải chỉ định tham số stratify điều này sẽ ràng buộc chặt chẽ hơn về mặt chia theo đúng tỉ lệ 75, 25 tương ứng với mỗi class trong tập train và tập test

print(y_train.value_counts())

print("---------------")

print(y_test.value_counts())


career_level
senior_specialist_or_project_manager      3469
manager_team_leader                       2138
bereichsleiter                             768
director_business_unit_leader               56
specialist                                  24
managing_director_small_medium_company       3
Name: count, dtype: int64
---------------
career_level
senior_specialist_or_project_manager      868
manager_team_leader                       534
bereichsleiter                            192
director_business_unit_leader              14
specialist                                  6
managing_director_small_medium_company      1
Name: count, dtype: int64


In [38]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6458 entries, 7848 to 1686
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        6458 non-null   object
 1   location     6458 non-null   object
 2   description  6458 non-null   object
 3   function     6458 non-null   object
 4   industry     6458 non-null   object
dtypes: object(5)
memory usage: 302.7+ KB


> Nhận xét: Dường như có không công bằng ở đây, dữ liệu của chúng ta rơi vào trường hợp imblance Data do đó giải pháp được đưa ra
>           đó là oversampling
> 
> **C2: chúng ta gộp các class bé lại với nhau, tuy nhiên cần chú ý khi gộp đó là nó phải nhất quán về mặt ý nghĩa, do đó nên gộp class managing_director_small_medium_company  và director_business_unit_leader thành 1 class (Do mức độ specialist có sự khác nhau giữa 2 class này)**

In [40]:
from imblearn.over_sampling import SMOTEN, RandomOverSampler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile



ros = SMOTEN(random_state=42, sampling_strategy={
    "director_business_unit_leader" : 500,
    "specialist" :500,
    "managing_director_small_medium_company": 500
}, k_neighbors=2)
X_train, y_train = ros.fit_resample(X_train, y_train)



preprocesser = ColumnTransformer(transformers=[
    ("title_feature", TfidfVectorizer(), "title"),
    ("location_feature", OneHotEncoder(handle_unknown='ignore'), ["location"]),
    ("description_feature", TfidfVectorizer(ngram_range=(1, 2), stop_words="english", min_df=0.01, max_df=0.95), "description"),
    ("function_feature", OneHotEncoder(handle_unknown='ignore'), ["function"]),
    ("industry_feature", TfidfVectorizer(), "industry")
])

cls = Pipeline(steps=[
    ("Processer", preprocesser),
    ("feature_selection", SelectKBest(chi2, k=500)),
    ("model", RandomForestClassifier())
])

params = {
    "model__n_estimators": [50, 100, 200],
    # "model__criterion": ["entropy", "gini", "log_loss"],
    "feature_selection__k": [i*100 for i in range(1, 11)]
}

grid_cls = GridSearchCV(cls, param_grid=params, cv=6, verbose=2, scoring="f1_weighted", n_jobs=-1)
grid_cls.fit(X_train, y_train)
print(grid_cls.best_params_)
print(grid_cls.best_score_)
y_predict = grid_cls.predict(X_test)
print(classification_report(y_test, y_predict)) 

